In [263]:
import numpy
import random
import copy
import math

class NQueensProblem:
    
    def __init__(self, n):
        self.__n = n
        self.__Q = n * 100
        self.__board = numpy.full((n, n), self.__Q, dtype=float)
        self.__last_board = copy.deepcopy(self.__board)

    def solve(self):
        ants = math.floor(self.__n / 2)
        self.__ant_colony(ants)
        
    #Esse método sintetiza o funcionamento do algoritmo de colonia de formigas
    #1 - Coloca formigas para andar pelo tabuleiro
    #2 - Calcula as quantidades de feromonio para os caminhos das formigas
    #3 - Atualiza os caminhos percorridos com o feromonio
    #4 - Executa os passos anteriores novamente ate o tabuleiro convergir ou atingir a quantidade máxima de iterações
    def __ant_colony(self, ants):
        for i in range(100):
            if self.__converged():
                break
            self.__last_board = copy.deepcopy(self.__board)
            for ant in range(ants):
                path = self.__run_ant_on_board()
                pheromone = self.__calculate_pheromone(path)
                self.__update_pheromone(path, pheromone)
        print("iterations:", i)
                
    #Esse método faz uma formiga andar pelo tabuleiro
    def __run_ant_on_board(self):
        path = []
        for column in range(self.__n):
            total = 0
            positions = []
            wheights = []
            for line in range(self.__n):
                if line not in set(path):
                    total += self.__board[line][column]
            for line in range(self.__n):
                if line not in set(path):
                    positions.append(line)
                    wheights.append(self.__board[line][column] / total)
            position = random.choices(positions, wheights)[0]
            path.append(position)
        return path
        
    #Esse método calcula a quantidade de feromonio para um determinado caminho
    def __calculate_pheromone(self, path):
        conflicts = self.__calculate_conflicts(path)
        if conflicts == 0:
            return self.__Q
        conflicts = conflicts / self.__n
        return self.__Q / conflicts
    
    #Esse método calcula quantos conflitos há em um caminho
    def __calculate_conflicts(self, path):
        board = numpy.zeros(shape=(self.__n, self.__n))
        for i in range(self.__n):
            for j in range(self.__n):
                if path[j] == i:
                    board[i][j] = 1
        conflicts = self.__get_board_conflicts(board)
        return conflicts
        
    #Esse método atualiza o feromonio no caminho que uma formiga percorreu
    def __update_pheromone(self, path, pheromone):
        portion = pheromone / self.__n
        evaporation = portion * 0.1
        for column in range(self.__n):
            line = path[column]
            self.__board[line][column] += portion
        for i in range(self.__n):
            for j in range(self.__n):
                path_line = path[column]
                if path_line != i:
                    self.__board[i][j] -= evaporation 
                    if self.__board[i][j] < self.__Q:
                        self.__board[i][j] = self.__Q
    
    #Esse método avalia se o tabuleiro convergiu para uma solução
    def __converged(self):
        difference = 0
        for i in range(self.__n):
            for j in range(self.__n):
                current = self.__board[i][j]
                last = self.__last_board[i][j]
                difference += (current - last)
        return (difference >= (self.__Q - 1)) and (difference <= (self.__Q + 1))
    
    #Esse método calcula todos os conflitos que ocorrem no tabuleiro
    def __get_board_conflicts(self, board):
        vertical = 0
        horizontal = 0
        diagonal = 0
        for i in range(self.__n):
            for j in range(self.__n):
                if board[i][j] == 1:
                    vertical += self.__get_vertical_conflicts(i, j, board)
                    horizontal += self.__get_horizontal_conflicts(i, j, board)
                    diagonal += self.__get_diagonal_conflicts(i, j, board)
        return vertical + horizontal + diagonal
    
    #Esse método calcula todos os conflitos que ocorrem na mesma coluna
    #do elemento no tabuleiro
    def __get_vertical_conflicts(self, i, j, board):
        conflicts = 0
        for line in range(self.__n):
            if line != i and board[line][j] == 1:
                conflicts += 1
        return conflicts
    
    #Esse método calcula todos os conflitos que ocorrem na mesma linha
    #do elemento no tabuleiro
    def __get_horizontal_conflicts(self, i, j, board):
        conflicts = 0
        for column in range(self.__n):
            if column != j and board[i][column] == 1:
                conflicts += 1
        return conflicts
    
    #Esse método calcula todos os conflitos que ocorrem nas diagonais
    #do elemento no tabuleiro
    def __get_diagonal_conflicts(self, i, j, board):
        top_right = self.__get_top_right_conflicts(i, j, board)
        top_left = self.__get_top_left_conflicts(i, j, board)
        bottom_right = self.__get_bottom_right_conflicts(i, j, board)
        bottom_left = self.__get_bottom_left_conflicts(i, j, board)
        conflicts = top_right + top_left + bottom_right + bottom_left
        return conflicts
    
    def __get_top_right_conflicts(self, i, j, board):
        conflicts = 0
        line = i - 1
        column = j + 1
        while line >= 0 and column < self.__n:
            if board[line][column] == 1:
                conflicts += 1
            column += 1
            line -= 1
        return conflicts
    
    def __get_top_left_conflicts(self, i, j, board):
        conflicts = 0
        line = i - 1
        column = j - 1
        while line >= 0 and column >= 0:
            if board[line][column] == 1:
                conflicts += 1
            column -= 1
            line -= 1
        return conflicts
    
    def __get_bottom_right_conflicts(self, i, j, board):
        conflicts = 0
        line = i + 1
        column = j + 1
        while line < self.__n and column < self.__n:
            if board[line][column] == 1:
                conflicts += 1
            column += 1
            line += 1
        return conflicts
    
    def __get_bottom_left_conflicts(self, i, j, board):
        conflicts = 0
        line = i + 1
        column = j - 1
        while line < self.__n and column >= 0:
            if board[line][column] == 1:
                conflicts += 1
            column -= 1
            line += 1
        return conflicts
    
    def __get_queens_board(self):
        copy_board = copy.deepcopy(self.__board)
        for column in range(self.__n):
            max_line = 0
            for line in range(1, self.__n):
                if copy_board[line][column] >= copy_board[max_line][column]:
                    copy_board[max_line][column] = 0
                    max_line = line
                else: 
                    copy_board[line][column] = 0
            if copy_board[max_line][column] > self.__Q:
                copy_board[max_line][column] = 1
            else:
                copy_board[max_line][column] = 0
        return copy_board
    
    def print_board(self):
        for i in range(self.__n):
            print(self.__board[i])    
            
    def print_solution(self):
        queens_board = self.__get_queens_board()
        queens_conflicts = self.__get_board_conflicts(queens_board)
        print("conflicts:", queens_conflicts)
        #for i in range(self.__n):
        #    print(queens_board[i])

In [265]:
problem = NQueensProblem(100)
problem.solve()
problem.print_solution()

iterations: 99
conflicts: 176
